In [1]:
import os
import pandas as pd
from datetime import datetime
import functools as ft
from tqdm import tqdm
import math

In [2]:
dataFiles = os.listdir("raw-data")
dataFiles.sort()

In [3]:
csvs = ["raw-data/{}".format(tmp) for tmp in [
    '2017-2018-Premier-League-Scores-and-Fixtures',
    '2018-2019-Premier-League-Scores-and-Fixtures',
    '2019-2020-Premier-League-Scores-and-Fixtures',
    '2020-2021-Premier-League-Scores-and-Fixtures',
    '2021-2022-Premier-League-Scores-and-Fixtures',
    '2022-2023-Premier-League-Scores-and-Fixtures',
]]

allGames = pd.concat([pd.read_csv(csv) for csv in csvs])[["Date", "Home", "Away"]]
allGames

,Date,Home,Away
0,2017-08-11,Arsenal,Leicester City
1,2017-08-12,Watford,Liverpool
2,2017-08-12,Crystal Palace,Huddersfield
3,2017-08-12,West Brom,Bournemouth
4,2017-08-12,Chelsea,Burnley
...,...,...,...
443,2022-05-22,Norwich City,Tottenham
444,2022-05-22,Manchester City,Aston Villa
445,2022-05-22,Liverpool,Wolves
446,2022-05-22,Burnley,Newcastle Utd


In [4]:
replaceNames = {
    "West Brom": "West Bromwich Albion",
    "Huddersfield": "Huddersfield Town",
    "Brighton": "Brighton and Hove Albion",
    "West Ham": "West Ham United",
    "Manchester Utd": "Manchester United",
    "Tottenham": "Tottenham Hotspur",
    "Newcastle Utd": "Newcastle United",
    "Wolves": "Wolverhampton Wanderers",
    "Sheffield Utd": "Sheffield United"
}


def isValidRow(row):
    return type(allGames.iloc[row]["Home"]) is str


def getTablesForRow(row):
    if not isValidRow(row):
        return 
    
    homeRaw = allGames.iloc[row]["Home"]
    awayRaw = allGames.iloc[row]["Away"]
    dateRaw = allGames.iloc[row]["Date"]
    
    if homeRaw in replaceNames:
        homeRaw = replaceNames[homeRaw]
    
    if awayRaw in replaceNames:
        awayRaw = replaceNames[awayRaw]
    
    home = homeRaw.replace(" ", "-")
    away = awayRaw.replace(" ", "-")
    date = datetime.strptime(dateRaw, '%Y-%m-%d').strftime("%B-%-d-%Y")
    
    substr = "{}-{}-{}".format(home, away, date)
    fileList = ["raw-data/{}".format(tmp) for tmp in filter(lambda x: substr in x, dataFiles)]
    fileList.sort(key = lambda x: int(x.split("-")[-1]))
    return [pd.read_csv(file) for file in fileList]



In [5]:
def getStat(columnName, columnKey, statName, homeTableIndex, tables, players, isNonNumeric = False):
    homeTable = tables[homeTableIndex]
    awayTable = tables[homeTableIndex + 7]
    
    assert homeTableIndex <= 9
    assert homeTable.iloc[0][columnName] == columnKey
    assert awayTable.iloc[0][columnName] == columnKey
    
    homeTableEnd = len(homeTable) - 1
    awayTableEnd = len(awayTable) - 1
    if homeTableIndex == 9:
        homeTableEnd += 1
        awayTableEnd += 1
    
    for i in range(1, homeTableEnd):
        row = homeTable.iloc[i]
        player = row["Unnamed: 0_level_0"]
        if player not in players:
            players[player] = dict()
        if isNonNumeric:
            players[player][statName] = row[columnName]
        else:   
            players[player][statName] = float(row[columnName])
    
    for i in range(1, awayTableEnd):
        row = awayTable.iloc[i]
        player = row["Unnamed: 0_level_0"]
        if player not in players:
            players[player] = dict()
        if isNonNumeric:
            players[player][statName] = row[columnName]
        else:   
            players[player][statName] = float(row[columnName])


def processRowByIndex(index):
    if not isValidRow(index):
        return list()
    
    row = allGames.iloc[index]
    date = row["Date"]
    tables = getTablesForRow(index)
    players = dict()
    
    getStat("Unnamed: 5_level_0", "Min", "Minutes", 3, tables, players)
    getStat("Unnamed: 3_level_0", "Pos", "Position(s)", 3, tables, players, isNonNumeric = True)
    getStat("Performance", "Gls", "Goals", 3, tables, players)
    getStat("Performance.1", "Ast", "Assists", 3, tables, players)
    getStat("Performance.4", "Sh", "Shots Total", 3, tables, players)
    getStat("Performance.5", "SoT", "Shots on Target", 3, tables, players)
    getStat("SCA", "SCA", "Shot Creating Actions", 3, tables, players)
    getStat("SCA.1", "GCA", "Goal Creating Actions", 3, tables, players)
    getStat("SCA.1", "GCA", "Goal Creating Actions", 3, tables, players)
    getStat("Unnamed: 23_level_0", "KP", "Assisted Shots", 4, tables, players)
    getStat("Unnamed: 24_level_0", "1/3", "Passes Into Attacking Third", 4, tables, players)
    getStat("Unnamed: 25_level_0", "PPA", "Passes Into Attacking Penalty", 4, tables, players)
    getStat("Unnamed: 26_level_0", "CrsPA", "Crosses Into Attacking Penalty", 4, tables, players)
    getStat("Touches", "Touches", "Touches", 7, tables, players)
    getStat("Touches.1", "Def Pen", "Defensive Penalty Touches", 7, tables, players)
    getStat("Touches.2", "Def 3rd", "Defensive Third Touches", 7, tables, players)
    getStat("Touches.3", "Mid 3rd", "Middle Third Touches", 7, tables, players)
    getStat("Touches.4", "Att 3rd", "Attacking Third Touches", 7, tables, players)
    getStat("Touches.5", "Att Pen", "Attacking Penalty Touches", 7, tables, players)
    getStat("Dribbles", "Succ", "Dribbles Successful", 7, tables, players)
    getStat("Dribbles.1", "Att", "Dribbles Attempted", 7, tables, players)
    getStat("Performance.10", "Int", "Interceptions", 3, tables, players)
    getStat("Performance.11", "Blocks", "Blocks", 3, tables, players)    
    getStat("Passes.3", "Prog", "Progressive Passes Completed", 3, tables, players)
    getStat("Passes.1", "Att", "Passes Attempted", 3, tables, players)
    getStat("Total.3", "TotDist", "Total Passing Distance", 4, tables, players)
    getStat("Total.4", "PrgDist", "Progressive Passing Distance", 4, tables, players)
    getStat("Pass Types.5", "Crs", "Corner Kicks", 5, tables, players)
    getStat("Pass Types.7", "CK", "Crosses", 5, tables, players)
    getStat("Passes", "Cmp", "Pass Completed", 3, tables, players)
    getStat("Short", "Cmp", "Short Passes Completed", 4, tables, players)
    getStat("Short.1", "Att", "Short Passes Attempted", 4, tables, players)
    getStat("Medium", "Cmp", "Medium Passes Completed", 4, tables, players)
    getStat("Medium.1", "Att", "Medium Passes Attempted", 4, tables, players)
    getStat("Long", "Cmp", "Long Passes Completed", 4, tables, players)
    getStat("Long.1", "Att", "Long Passes Attempted", 4, tables, players)
    getStat("Tackles", "Tkl", "Tackles", 6, tables, players)
    getStat("Tackles.1", "TklW", "Tackles Won", 6, tables, players)
    getStat("Performance.3", "Fls", "Fouls", 8, tables, players)
    getStat("Performance.12", "Recov", "Loose Ball Recoveries", 8, tables, players)
    getStat("Aerial Duels", "Won", "Aerials Won", 8, tables, players)
    getStat("Aerial Duels.1", "Lost", "Aerials Lost", 8, tables, players)
    getStat("Shot Stopping", "SoTA", "Shots on Target Against", 9, tables, players)
    getStat("Shot Stopping.1", "GA", "Goals Against", 9, tables, players)
    
    for player in players:
        players[player]["Player"] = player
        players[player]["Date"] = date
    
    assert len(players) >= 22
    nanMinutesPlayers = list()
    for player in players:
        if math.isnan(players[player]["Minutes"]):
            nanMinutesPlayers.append(player)
    
    for player in nanMinutesPlayers:
         del players[player]
    
    totalMinutes = 0
    for player in players:
        totalMinutes += players[player]["Minutes"]
    assert totalMinutes <= 90 * 11 * 2, "row on index {} is funky".format(index)
    
    return list(players.values())

In [6]:
allAppearances = list()

for index in tqdm(range(len(allGames))):
    allAppearances.extend(processRowByIndex(index))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2603/2603 [04:27<00:00,  9.74it/s]


In [7]:
allAppearances.sort(key = lambda x: (x["Player"], x["Date"]))
df = pd.DataFrame(allAppearances)
newColumns = list(df.columns)
newColumns.remove("Player")
newColumns.remove("Date")
newColumns = ["Player", "Date"] + newColumns
df = df[newColumns]
df
assert len(df["Minutes"].unique()) <= 90
assert float("nan") not in df["Minutes"]

In [8]:
df.to_csv("player-data", index=False)

In [11]:
df = pd.read_csv("player-data")
df

,Player,Date,Minutes,Position(s),Goals,Assists,Shots Total,Shots on Target,Shot Creating Actions,Goal Creating Actions,...,Long Passes Completed,Long Passes Attempted,Tackles,Tackles Won,Fouls,Loose Ball Recoveries,Aerials Won,Aerials Lost,Shots on Target Against,Goals Against
0,Aaron Connolly,2019-08-31,25.0,FW,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,NaN,NaN
1,Aaron Connolly,2019-09-14,7.0,FW,0.0,0.0,2.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,NaN,NaN
2,Aaron Connolly,2019-09-21,21.0,"FW,LW",0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,NaN,NaN
3,Aaron Connolly,2019-09-28,26.0,FW,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,NaN,NaN
4,Aaron Connolly,2019-10-05,79.0,FW,2.0,0.0,6.0,3.0,4.0,1.0,...,0.0,0.0,0.0,0.0,1.0,3.0,4.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62899,Łukasz Fabiański,2022-05-08,90.0,GK,0.0,0.0,0.0,0.0,0.0,0.0,...,4.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0
62900,Łukasz Fabiański,2022-05-15,90.0,GK,0.0,0.0,0.0,0.0,1.0,0.0,...,14.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,2.0
62901,Łukasz Fabiański,2022-05-15,90.0,GK,0.0,0.0,0.0,0.0,1.0,0.0,...,14.0,27.0,0.0,0.0,0.0,1.0,0.0,0.0,8.0,2.0
62902,Łukasz Fabiański,2022-05-22,90.0,GK,0.0,0.0,0.0,0.0,0.0,0.0,...,8.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,3.0


In [ ]:
allPlayers = df["Player"].unique()
tmp = list()
for player in tqdm(allPlayers):
    uniquePos = df[df["Player"] == player]["Position(s)"].unique()
    allPos = set()
    for p in uniquePos:
        allPos.update(p.split(","))
    tmp.append({"Player": player, "Position(s)": list(allPos)})
pd.DataFrame(tmp).to_csv("all-players", index=False)
allPlayers = pd.read_csv("all-players")
allPlayers

 80%|█████████████████████████████████████████████████████████████████████████████████████████████████▎                       | 904/1124 [00:03<00:00, 260.92it/s]